In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_csv('Dataset.csv', sep=',')
df = df.drop(df.columns[0],axis=1)
X = df[df.columns[:53]]
Y = df[['Target']]
X = normalize(X,Y,X.shape[0])
#X = norm(X)


In [6]:
#X[X.columns[36:38]]


max_values_ind = Y.copy().sort_values('Target',ascending=False).head(49).index
Y = Y.drop(Y.index[max_values_ind])
X = X.drop(X.index[max_values_ind])
#Y.sort_values('Target',ascending=False).head(100)
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)
Y.shape

(40900, 1)

In [7]:
def normalize(X,Y,nrows):
    BinaryFeatures = ['Post Promotion Status','published_weekday_0','published_weekday_1','published_weekday_2',
                      'published_weekday_3', 'published_weekday_4','published_weekday_5','published_weekday_6',
                     'base_weekday_0','base_weekday_1','base_weekday_2','base_weekday_3','base_weekday_4',
                      'base_weekday_5','base_weekday_6']
    BF = X[BinaryFeatures] 
    
    DecimalFeatures = ['Page Popularity','Page Checkins','Page talking about',
                       'extra_0','extra_1','extra_2','extra_3','extra_4','extra_5',
                       'extra_6','extra_7','extra_8','extra_9','extra_10',
                       'extra_11','extra_12','extra_13','extra_14','extra_15','extra_16',
                       'extra_17','extra_18','extra_19','extra_20','extra_21','extra_22','extra_23','extra_24',
                       'CC1','CC2','CC3','CC4','CC5','Base Time','Post Length','Post Share Count','H Local']
    DF = X[DecimalFeatures]
    E = DF.mean()
    D = DF.std()
    DF = (DF-E)/D
    
    # 'H Local' feature's modification
    #HLocal = Y['Target']/X['H Local']#comments/an hour 
    #CategoryFeature =['Page Category']
    #CF = X[CategoryFeature]
    
    frequency = X['Page Category'].value_counts(sort=False) #frequency of categories
    CF = 1/frequency[X['Page Category']]
    CF = CF.reset_index(drop=True)
    
    # x0 = [1,1,...1] fpr b koeficient
    x1 = pd.DataFrame({'feature 1': np.ones(nrows)})

    result = pd.concat([x1,DF,CF, BF], axis=1) #,X['H Local']
    return result



In [8]:
def gradient(X,Y,W,lyambda, n, nfeat):
    Ypred = np.dot(X,W)
    #print(Y[:10], Ypred[:10])
    dy = Y - Ypred
    cost = np.sum(dy**2)/n
    #print( "cost", cost)
    gr = np.dot(X.T,dy)/n
    Wnew = W + 2* lyambda * gr.reshape(nfeat,1)
    return Wnew

def foo_RMSE(Xt, Yt, W):
    rmse = math.sqrt(np.sum(((Yt-np.dot(Xt,W))**2))/len(Yt))
    return rmse

def foo_R2(X,Y,W):
    Ey = Y.mean()
    r2 = 1 - np.sum((np.dot(X,W)-Y)**2)/np.sum((Y - Ey)**2)
    return r2

In [9]:
nfolds = 5
indexes = np.arange(X.shape[0]//nfolds * nfolds)
indexes = indexes.reshape(X.shape[0]//nfolds,5)
li =[] 
for i in range(nfolds):
    li.append(indexes[:,i].tolist())
Xfold =[]
Yfold =[]
for ind in li:
    Xfold.append(X.loc[ind])
    Yfold.append(Y.loc[ind])
   

In [10]:
nsteps = 500
e = 10e-5

lyambda = 0.05
table = pd.DataFrame({})
nfeatures = X.shape[1]
chunk = 0
for i in range(nfolds):
    
    Xtrain = Xfold[i]
    Xtest = Xfold[(i+1)%5]
    Ytrain = Yfold[i]
    Ytest = Yfold[(i+1)%5]
    
    nrows = Xtrain.shape[0]
    chunk+=1
    k = 0
    W = np.ones(nfeatures).reshape(nfeatures,1)
    while True:
        k+=1
        Wnew = gradient(Xtrain,Ytrain,W,lyambda/math.sqrt(k), nrows, nfeatures)
        estimation = np.max(abs(W - Wnew))
    
        if (estimation < e)or(k>nsteps):
            break
        W = Wnew
        
    print("CHUNK # ", chunk)    
    R2train = foo_R2(Xtrain,Ytrain,W)
    print ("R2 for train: ",R2train[0] , "\n")

    RMSEtrain = foo_RMSE(Xtrain,Ytrain,W)
    print ("RMSE for train: ",RMSEtrain , "\n") 
 
    
    #test
    R2test = foo_R2(Xtest,Ytest,W)
    print ("R2 for test: ",R2test[0] , "\n")

    RMSEtest = foo_RMSE(Xtest,Ytest,W)
    print ("RMSE for test: ",RMSEtest , "\n")
   
    
    a = [R2train[0],RMSEtrain,R2test[0],RMSEtest]
    
    for feature in W.tolist():
        a.append(feature[0])
    col = pd.DataFrame({'chunk %d'%chunk: a })
    table =  pd.concat([table, col],axis = 1)


table.index=['R2_train', 'RMSE_train', 'R2_test', 'RMSE_test','b','Page Popularity','Page Checkins',
             'Page talking about','extra_0','extra_1','extra_2','extra_3','extra_4','extra_5','extra_6',
             'extra_7','extra_8','extra_9','extra_10','extra_11','extra_12','extra_13','extra_14','extra_15',
             'extra_16','extra_17','extra_18','extra_19','extra_20','extra_21','extra_22','extra_23','extra_24',
             'CC1','CC2','CC3','CC4','CC5','Base Time','Post Length','Post Share Count','H Local','Page Category', 
             'Post Promotion Status','published_weekday_0','published_weekday_1','published_weekday_2',
             'published_weekday_3', 'published_weekday_4','published_weekday_5','published_weekday_6',
             'base_weekday_0','base_weekday_1','base_weekday_2','base_weekday_3','base_weekday_4',
             'base_weekday_5','base_weekday_6']
table.index.name = 'Features'
table =  pd.concat([table, pd.DataFrame({'E': table.mean(axis = 1) })],axis = 1)
table =  pd.concat([table, pd.DataFrame({'STD': table.std(axis = 1) })],axis = 1)
table.to_csv('table.csv')   

CHUNK #  1
R2 for train:  0.41400440264 

RMSE for train:  18.77310533355504 

R2 for test:  0.345862524114 

RMSE for test:  20.037095264073113 

CHUNK #  2
R2 for train:  0.365005787421 

RMSE for train:  19.741726508912002 

R2 for test:  0.398070902212 

RMSE for test:  18.259225682487035 

CHUNK #  3
R2 for train:  0.42237829859 

RMSE for train:  17.886750050819256 

R2 for test:  0.397786436894 

RMSE for test:  20.890970307927788 

CHUNK #  4
R2 for train:  0.426365847146 

RMSE for train:  20.389230975584578 

R2 for test:  0.419270240452 

RMSE for test:  20.214093537826937 

CHUNK #  5
R2 for train:  0.432217070214 

RMSE for train:  19.987496269799667 

R2 for test:  0.378169952695 

RMSE for test:  19.33858938666766 

